In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
## read document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

## divide docs into chunks
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [3]:
doc=read_doc('documents/')
len(doc)

104

In [4]:
documents=chunk_data(docs=doc)
len(documents)

104

In [5]:
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
vectors=embeddings.embed_query("What's up")
len(vectors)

/Users/rafi/Projects/langchain-proj/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


1536

In [6]:
from pinecone import Pinecone
## Vector Search DB in Pinecone
pinecone_client=Pinecone(
    api_key=os.environ['PINECONE_API_KEY'],
    environment="gcp-starter"
)
index_name="langchainvector"

In [7]:
from langchain.vectorstores import Pinecone as PineconeDB
index=PineconeDB.from_documents(doc, embeddings, index_name=index_name)

In [8]:
## Cosine Similarity Retrieve Results
def retrieve_query(query, k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [9]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model_name="gpt-4-0125-preview",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

/Users/rafi/Projects/langchain-proj/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [12]:
our_query = "What is hypertrophy and what are the quads?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='JEFF NIPPARD  FUNDAMENTALS HYPERTROPHY PROGRAM12\nFUNCTIONAL ANATOMY\nIt is important to understand the functional anatomy and biomechanics of the \nmain muscles we’ll be targeting before we can understand how to best train them. \nFunctional anatomy determines what muscles can do. There are two things to \nconsider when looking at a muscle’s functional anatomy – its origin and insertion. \nMuscles attach to bone by tendons from at least two points. The origin is the fixed \nattachment which does not move and the insertion is the attachment which moves \ncloser to the origin when a muscle contracts. This is the contracting phase, referred \nto as the concentric phase (also known as the “positive” phase), which is normally \nfollowed by the eccentric phase (lowering the weight – also known as the “negative” \nphase).\nQUADRICEPS: The quadriceps (“quads” for short) are \ncomprised of four muscles, often referred to as “heads”: \nthe vastus lateralis (“quad sweep”)

In [13]:
our_query = "Can you tell me about Jeff Nippard's background?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content='JEFF NIPPARD  FUNDAMENTALS HYPERTROPHY PROGRAM5\nABOUT ME\nJeff is a professional drug-free bodybuilder and powerlifter. Through his science-\nbased Youtube channel which has gathered an audience of millions of subscribers, \nJeff aims to share the knowledge he has gathered through university education and \nfield experience with others who are passionate about the science behind building \nmuscle, losing fat and getting healthier.\nHe earned the title of Mr. Junior Canada for natural bodybuilding in 2012 and as a \npowerlifter, Jeff held the Canadian national record for the bench press in 2014. As a \npowerlifter, Jeff has claimed a 502 lb squat, 336 lb bench press and a 518 lb deadlift \nwith an all time best Wilks score of 446.', metadata={'page': 4.0, 'source': 'documents/Fundamentals_Hypertrophy_Program.pdf'}), Document(page_content='JEFF NIPPARD  FUNDAMENTALS HYPERTROPHY PROGRAM5\nABOUT ME\nJeff is a professional drug-free bodybuilder and powerlifter. Thro